In [1]:
import numpy as np
import pandas as pd
import csv as csv
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from sklearn import datasets, svm, metrics

In [2]:
in_file = 'train.csv'
locations = pd.read_csv(in_file)

In [3]:
locations_rounded = locations.round({'x': 1, 'y': 1})
locations_sort = locations_rounded.sort_values(['x' , 'y'])
#locations_d = locations_rounded[(locations_rounded['x'] == 1.1) & (locations_rounded['y'] == 1.1)]

In [15]:
locations_sort_slice = locations_sort[(locations_sort['x'] >= 1.0) & (locations_sort['x'] <= 2.0) & \
                                     (locations_sort['y'] >= 1.0) & (locations_sort['y'] <= 2.0)]

locations_sort_slice.loc[:, 'minute'] = (locations_sort_slice['time']) % 1440
locations_sort_slice.loc[:, 'hour'] = ((locations_sort_slice['time']) % 1440)/60
locations_sort_slice.loc[:, 'weekday'] = ((locations_sort_slice['time']) /(60*24)) % 7 
locations_sort_slice.loc[:, 'month'] = ((locations_sort_slice['time']) /(60*24*30)) % 12
locations_sort_slice.loc[:, 'year'] = ((locations_sort_slice['time']) /(60*24*365))
locations_sort_slice.loc[:, 'day'] = ((locations_sort_slice['time'])/(60*24)) % 365

#locations_sort_slice = locations_sort_slice[['row_id', 'x', 'y', 'accuracy', 'time', 'greater_than_800', 'place_id']]
locations_sort_slice = locations_sort_slice[['row_id', 'x', 'y', 'hour', 'weekday', 'month', 'year', 'place_id']]
locations_sort_slice = locations_sort_slice.round({'hour': 0, 'weekday': 0, 'month': 0, 'year': 0,})
#locations_sort_slice = locations_sort_slice.drop('time', 1)
locations_sort_slice

,row_id,x,y,hour,weekday,month,year,place_id
2636,2636,1,1,16,4,6,1,5152560199
3164,3164,1,1,18,3,12,1,5152560199
6245,6245,1,1,5,3,4,0,2013736336
8819,8819,1,1,18,3,6,1,2924413804
8909,8909,1,1,7,0,5,0,3546958533
18156,18156,1,1,6,1,2,1,2398651098
25663,25663,1,1,3,3,5,1,7995458948
29380,29380,1,1,5,3,5,0,3303345536
32253,32253,1,1,12,7,8,1,1195596952
36022,36022,1,1,19,3,3,1,9787013492


In [16]:
from sklearn.cross_validation import train_test_split

locations_train, locations_test = train_test_split(locations_sort_slice, test_size = 0.3)

n_samples = locations_sort_slice.shape[0]
feature_cols = list(locations_train.columns[0:7])  
target_col = locations_train.columns[-1]

X_train = locations_train[feature_cols]  # feature values 
y_train = locations_train[target_col]  # corresponding targets/labels
X_test = locations_test[feature_cols]  
y_test = locations_test[target_col] 
X_train

,row_id,x,y,hour,weekday,month,year
10312552,10312552,1.0,2.0,12,5,6,0
17372944,17372944,1.7,1.7,20,2,11,1
548490,548490,2.0,1.9,2,0,5,1
15413303,15413303,1.4,1.0,0,5,7,1
23229949,23229949,2.0,1.4,3,0,6,1
24165847,24165847,1.1,1.7,18,4,12,1
1472416,1472416,1.5,1.3,1,3,4,0
7742650,7742650,1.7,1.0,22,5,6,0
24893687,24893687,1.8,1.7,9,2,5,0
6931423,6931423,1.9,1.8,17,1,2,1


In [7]:
#locations_c = locations_train.round({'x': 1, 'y': 1})
#locations_d = locations_c[(locations_c['x'] == 1.1) & (locations_c['y'] == 1.1)]
#locations_time = locations_sort_slice['time'] % 1440
#locations_time
#locations_d.loc[(locations_d['time'] > 0), 'time'] = locations_d2
locations_sort_slice.groupby('place_id').count()
#test = locations_sort_slice[(locations_sort_slice['place_id'] == 1673534850)]
#locations_d.sort_values(['time'])
#test.groupby('time').count()
#test[(test['time'] < 800)]
#test.loc[:, 'greater_than_800'] = np.where(test['time'] > 800, 1, 0)
#test_count = test.groupby('greater_than_800').count()
#test_count[['row_id']].plot(kind='bar')

,row_id,x,y,hour,weekday,month,year
place_id,,,,,,,
1000213704,4,4,4,4,4,4,4
1000842315,5,5,5,5,5,5,5
1003598119,62,62,62,62,62,62,62
1003835757,98,98,98,98,98,98,98
1005236037,2,2,2,2,2,2,2
1006316884,768,768,768,768,768,768,768
1008473838,1,1,1,1,1,1,1
1009014488,2,2,2,2,2,2,2
1010803518,1,1,1,1,1,1,1


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [18]:
clf = DecisionTreeClassifier(min_samples_split=100)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=100, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [19]:
pred = clf.predict(X_test)

In [20]:
acc = accuracy_score(pred, y_test)
print acc

0.313866162803
